In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [3]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '09-11-2021' \
                    AND div_liquida < '0' \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,ABEV3,"-R$ 16.973.000.000,00","R$ 17,39",17.27,5.51,12.41,23.4,"R$ 15.849.700.000,00","R$ 3.552.530.000,00"
1,SULA11,"-R$ 14.366.600.000,00","R$ 27,60",28.93,19.39,-14.65,2.1,"R$ 406.221.000,00","R$ 280.319.000,00"
2,SULA3,"-R$ 14.366.600.000,00","R$ 9,53",29.97,6.46,-12.29,2.1,"R$ 406.221.000,00","R$ 280.319.000,00"
3,SULA4,"-R$ 14.366.600.000,00","R$ 9,08",28.56,6.46,-15.51,2.1,"R$ 406.221.000,00","R$ 280.319.000,00"
4,B3SA3,"-R$ 10.022.700.000,00","R$ 12,47",16.17,3.69,8.75,45.8,"R$ 4.722.880.000,00","R$ 1.176.130.000,00"
5,PSSA3,"-R$ 8.465.820.000,00","R$ 22,96",10.46,14.03,0.39,6.9,"R$ 1.418.710.000,00","R$ 60.104.000,00"
6,CMIN3,"-R$ 7.392.710.000,00","R$ 5,93",4.73,2.55,2.02,33.0,"R$ 7.009.350.000,00","R$ 804.065.000,00"


In [4]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""

# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
